In [23]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random
import os
import joblib
import time

def simular_datos_iot(n=1000, sensores=3, intervalo_segundos=60, semilla=2025):
    np.random.seed(semilla)
    registros = []
    inicio = datetime.now()

    for sensor_id in range(1, sensores + 1):
        tiempo = np.arange(n)
        base_temp, base_hum, base_co2 = 28, 65, 420

        temperatura = base_temp + np.random.normal(0, 1.0, n) + 2 * np.sin(2 * np.pi * tiempo / 720)
        humedad = base_hum + np.random.normal(0, 3.0, n) - 0.005 * tiempo
        co2 = base_co2 + np.random.normal(0, 10, n)
        tipo_evento = np.array(["normal"] * n)

        # Estrés térmico
        bloque_inicio = n // 3
        bloque_fin = bloque_inicio + n // 6
        stress = (tiempo >= bloque_inicio) & (tiempo <= bloque_fin)
        temperatura[stress] += 6
        humedad[stress] -= 20
        co2[stress] += 80
        tipo_evento[stress] = "estres_termico"

        presion = np.random.normal(1012, 4, n)
        zona_critica = (((temperatura > 33.5) & (humedad < 50)) | (co2 > 500)).astype(int)

        # Ruido en etiquetas
        ruido = int(n * 0.02)
        idx_ruido = np.random.choice(n, ruido, replace=False)
        zona_critica[idx_ruido] = 1 - zona_critica[idx_ruido]

        timestamps = [
            inicio + timedelta(seconds=i * intervalo_segundos + random.randint(-5, 5))
            for i in range(n)
        ]

        df = pd.DataFrame({
            "timestamp": timestamps,
            "sensor_id": sensor_id,
            "temperatura": temperatura,
            "humedad": humedad,
            "co2": co2,
            "presion": presion,
            "zona_critica": zona_critica,
            "tipo_evento": tipo_evento
        })

        registros.append(df)

    return pd.concat(registros, ignore_index=True)

# Clase sensor con ajuste de frecuencia
class SensorAutoconfigurable:
    def __init__(self, sensor_id, modelo, data):
        self.id = sensor_id
        self.modelo = modelo
        self.data = data[data["sensor_id"] == sensor_id].reset_index(drop=True)
        self.frecuencia = 10  # segundos
        self.protocolo = "WiFi"
        self.bateria = 100  # %
        self.index = 0

    def leer_datos(self):
        if self.index >= len(self.data):
            return None
        fila = self.data.iloc[self.index]
        self.index += 1
        return fila["temperatura"], fila["humedad"], fila["co2"], fila["timestamp"]

    def predecir(self, temp, hum, co2):
        entrada = pd.DataFrame([[temp, hum, co2]], columns=["temperatura", "humedad", "co2"])
        return self.modelo.predict(entrada)[0]

    def autoconfigurar(self, zona_critica):
        if zona_critica:
            self.frecuencia = 2
            self.protocolo = "LoRaWAN"
        elif self.bateria < 30:
            self.frecuencia = 30
            self.protocolo = "LoRa"
        else:
            self.frecuencia = 10
            self.protocolo = "WiFi"

    def operar(self, ciclos=20):
        for _ in range(ciclos):
            lectura = self.leer_datos()
            if lectura is None:
                print("No hay más datos.")
                break

            temp, hum, co2, ts = lectura
            zona_critica = self.predecir(temp, hum, co2)
            self.autoconfigurar(zona_critica)
            self.bateria -= np.random.uniform(0.5, 2.0)

            print(f"[{ts}] Sensor {self.id} | T: {temp:.1f}°C | H: {hum:.1f}% | CO₂: {co2:.1f}ppm | "
                  f"{'CRITICAL' if zona_critica else 'Normal'} | Freq: {self.frecuencia}s | "
                  f"Protocol: {self.protocolo} | Battery: {self.bateria:.1f}%")
            time.sleep(0.2)

# Generar dataset
if __name__ == "__main__":
    os.makedirs("Datos", exist_ok=True)
    df = simular_datos_iot(n=3000, sensores=4)
    df.to_csv("Datos/sensores.csv", index=False)
    print("Dataset generado con eventos de estrés térmico.")

Dataset generado con eventos de estrés térmico.


# Generar esenario de exp

In [8]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# -----------------------------
# 1) Simulación
# -----------------------------
def simulate_thermal_stress_in_cattle_pen_aligned(n=720, sensor_id=5, interval_seconds=60, seed=42):
    np.random.seed(seed)
    start_time = datetime.now()
    time_indices = np.arange(n)

    base_temp = 28
    base_hum = 65
    base_co2 = 420

    temperatura = base_temp + np.random.normal(0, 1.0, size=n) + 2 * np.sin(2 * np.pi * time_indices / 720)
    humedad = base_hum + np.random.normal(0, 3.0, size=n) - 0.005 * time_indices
    co2 = base_co2 + np.random.normal(0, 10, size=n)
    tipo_evento = np.array(["normal"] * n)

    # Bloque de estrés térmico
    stress_start = 16
    stress_end = stress_start + n // 10
    stress_indices = (time_indices >= stress_start) & (time_indices <= stress_end)

    temperatura[stress_indices] += 6
    humedad[stress_indices] -= 20
    co2[stress_indices] += 80
    tipo_evento[stress_indices] = "estres_termico"

    # Zona crítica
    zona_critica = (((temperatura > 33.5) & (humedad < 50)) | (co2 > 500)).astype(int)

    # Tiempo con jitter
    timestamps = [
        start_time + timedelta(seconds=i * interval_seconds + random.randint(-5, 5))
        for i in range(n)
    ]

    df = pd.DataFrame({
        'timestamp': timestamps,
        'sensor_id': sensor_id,
        'temperatura': temperatura,
        'humedad': humedad,
        'co2': co2,
        'presion': np.random.normal(1012, 4, size=n),
        'zona_critica': zona_critica,
        'tipo_evento': tipo_evento
    })

    return df

# -----------------------------
# 2) Gráfico con tres variables y colores diferenciados
# -----------------------------
def plot_temp_co2_hum(df, out_path="Resultados/thermal_stress_plot_clean.png", title=None):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    fig, ax1 = plt.subplots(figsize=(14, 6))

    # Temperatura (rojo)
    l_temp, = ax1.plot(df['timestamp'], df['temperatura'], color='red', linewidth=1.5, label='Temperature (°C)')
    ax1.set_ylabel("Temperature (°C)", fontsize=14)
    ax1.tick_params(axis='y', labelsize=12)

    # CO₂ (verde)
    ax2 = ax1.twinx()
    l_co2, = ax2.plot(df['timestamp'], df['co2'], color='green', linewidth=1.5, label='CO₂ (ppm)')
    ax2.set_ylabel("CO₂ (ppm)", fontsize=14)
    ax2.tick_params(axis='y', labelsize=12)

    # Humedad (azul, en un tercer eje)
    ax3 = ax1.twinx()
    ax3.spines["right"].set_position(("axes", 1.10))  # desplazar eje extra
    l_hum, = ax3.plot(df['timestamp'], df['humedad'], color='blue', linewidth=1.5, linestyle='--', label='Humidity (%)')
    ax3.set_ylabel("Humidity (%)", fontsize=14)
    ax3.tick_params(axis='y', labelsize=12)

    # Eje X
    ax1.set_xlabel("Timestamp", fontsize=14)
    ax1.tick_params(axis='x', labelsize=12)

    # Zonas críticas
    mask = df['zona_critica'].eq(1)
    groups = (mask != mask.shift()).cumsum()
    first = True
    for _, g in df[mask].groupby(groups[mask]):
        start = g['timestamp'].iloc[0]
        end = g['timestamp'].iloc[-1]
        ax1.axvspan(start, end, color='red', alpha=0.12, lw=0,
                    label='Critical Zone' if first else None, zorder=0)
        first = False

    # Título
    if title is None:
        title = "Thermal Stress Simulation – Temperature, CO₂ and Humidity"
    fig.suptitle(title, fontsize=16)
    fig.autofmt_xdate(rotation=45)
    ax1.grid(True, alpha=0.3)

    # Leyenda fuera del gráfico (abajo, centrada)
    handles = [l_temp, l_co2, l_hum]
    labels = [h.get_label() for h in handles]
    if not first:
        handles.append(Patch(facecolor='red', alpha=0.12, label='Critical Zone'))
        labels.append('Critical Zone')
    fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.01), ncol=4, fontsize=12)

    plt.tight_layout()
    plt.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

# -----------------------------
# 3) Ejecutar simulación y graficar
# -----------------------------
if __name__ == "__main__":
    os.makedirs("Datos", exist_ok=True)
    df_sim = simulate_thermal_stress_in_cattle_pen_aligned()
    df_sim.to_csv("Datos/thermal_stress_simulation.csv", index=False)
    print("Dataset guardado en Datos/thermal_stress_simulation.csv")

    plot_temp_co2_hum(
        df_sim,
        out_path="Resultados/thermal_stress_plot_clean.png",
        title="Thermal Stress Simulation in Cattle (Temp–CO₂–Humidity)"
    )
    print("Figura guardada en Resultados/thermal_stress_plot_clean.png")

Dataset guardado en Datos/thermal_stress_simulation.csv
Figura guardada en Resultados/thermal_stress_plot_clean.png


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates

# -----------------------------
# 1) Simulación (igual a tu versión previa)
# -----------------------------
def simulate_thermal_stress_in_cattle_pen_aligned(n=720, sensor_id=5, interval_seconds=60, seed=42):
    np.random.seed(seed)
    start_time = datetime.now().replace(hour=11, minute=0, second=0, microsecond=0)  # 11:00 para reflejar ventana 11–15 h
    time_indices = np.arange(n)

    base_temp = 28
    base_hum = 65
    base_co2 = 420

    temperatura = base_temp + np.random.normal(0, 1.0, size=n) + 2 * np.sin(2 * np.pi * time_indices / 720)
    humedad = base_hum + np.random.normal(0, 3.0, size=n) - 0.005 * time_indices
    co2 = base_co2 + np.random.normal(0, 10, size=n)
    tipo_evento = np.array(["normal"] * n)

    # Bloque de estrés térmico
    stress_start = 16
    stress_end = stress_start + n // 10
    stress_indices = (time_indices >= stress_start) & (time_indices <= stress_end)

    temperatura[stress_indices] += 6
    humedad[stress_indices] -= 20
    co2[stress_indices] += 80
    tipo_evento[stress_indices] = "estres_termico"

    # Zona crítica (usa temp, hum, CO2)
    zona_critica = (((temperatura > 33.5) & (humedad < 50)) | (co2 > 500)).astype(int)

    # Timestamps (11:00 a ~ 15:00 aprox., con jitter)
    timestamps = [
        start_time + timedelta(seconds=i * interval_seconds + random.randint(-5, 5))
        for i in range(n)
    ]

    df = pd.DataFrame({
        'timestamp': timestamps,
        'sensor_id': sensor_id,
        'temperatura': temperatura,
        'humedad': humedad,
        'co2': co2,
        'presion': np.random.normal(1012, 4, size=n),
        'zona_critica': zona_critica,
        'tipo_evento': tipo_evento
    })

    return df

# -----------------------------
# 2) Formateo del eje X: solo hora (HH:MM), sin fecha
# -----------------------------
def format_time_axis(ax, start=None, end=None):
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    if start is not None and end is not None:
        ax.set_xlim(start, end)
    for label in ax.get_xticklabels():
        label.set_rotation(0)
        label.set_ha('center')

# -----------------------------
# 3) Figura combinada (Temp, Hum, CO2) con tipografías grandes
# -----------------------------
def plot_temp_hum_co2(df, out_path="Resultados/fig2_combined.png"):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    fig, ax1 = plt.subplots(figsize=(13, 6))  # tamaño más grande para legibilidad

    # Temperatura (rojo) - eje izquierdo
    l_temp, = ax1.plot(df['timestamp'], df['temperatura'], color='red', linewidth=2.0, label='Temperature (°C)')
    ax1.set_ylabel("Temperature (°C)", fontsize=18)
    ax1.tick_params(axis='y', labelsize=14)
    ax1.grid(True, alpha=0.3)

    # CO2 (verde) - primer eje derecho
    ax2 = ax1.twinx()
    l_co2, = ax2.plot(df['timestamp'], df['co2'], color='green', linewidth=2.0, label='CO₂ (ppm)')
    ax2.set_ylabel("CO₂ (ppm)", fontsize=18)
    ax2.tick_params(axis='y', labelsize=14)

    # Humedad (azul) - segundo eje derecho desplazado
    ax3 = ax1.twinx()
    ax3.spines['right'].set_position(('axes', 1.10))
    l_hum, = ax3.plot(df['timestamp'], df['humedad'], color='blue', linewidth=2.0, linestyle='--', label='Humidity (%)')
    ax3.set_ylabel("Humidity (%)", fontsize=18)
    ax3.tick_params(axis='y', labelsize=14)

    # Eje X (solo hora)
    ax1.set_xlabel("Time (HH:MM)", fontsize=18)
    ax1.tick_params(axis='x', labelsize=14)
    format_time_axis(ax1, df['timestamp'].min(), df['timestamp'].max())

    # Zonas críticas (sombreado)
    mask = df['zona_critica'].eq(1)
    groups = (mask != mask.shift()).cumsum()
    first = True
    for _, g in df[mask].groupby(groups[mask]):
        ax1.axvspan(g['timestamp'].iloc[0], g['timestamp'].iloc[-1],
                    color='red', alpha=0.12, lw=0,
                    label='Critical Zone' if first else None, zorder=0)
        first = False

    # LEYENDA grande y fuera (arriba, sin cubrir el gráfico)
    handles = [l_temp, l_hum, l_co2]
    labels = [h.get_label() for h in handles]
    if not first:
        handles.append(Patch(facecolor='red', alpha=0.12, label='Critical Zone'))
        labels.append('Critical Zone')

    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.02),
               ncol=4, fontsize=13)

    # Sin título interno (caption se maneja en LaTeX)
    plt.tight_layout()
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

# -----------------------------
# 4) Figura de temperatura sola (para Fig. 4 alternativa)
# -----------------------------
def plot_temperature_only(df, out_path="Resultados/fig4_temperature_only.png"):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    fig, ax = plt.subplots(figsize=(13, 5.5))

    ax.plot(df['timestamp'], df['temperatura'], color='red', linewidth=2.2, label='Temperature (°C)')
    ax.set_xlabel("Time (HH:MM)", fontsize=18)
    ax.set_ylabel("Temperature (°C)", fontsize=18)
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(True, alpha=0.3)

    # Eje X solo hora
    format_time_axis(ax, df['timestamp'].min(), df['timestamp'].max())

    # Marcar eventos críticos sobre la curva de temperatura
    criticos = df[df["zona_critica"] == 1]
    if not criticos.empty:
        ax.scatter(criticos["timestamp"], criticos["temperatura"],
                   color="black", marker='x', s=60, label="Critical Events")

    # Leyenda grande fuera (arriba)
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.02),
               ncol=3, fontsize=13)

    plt.tight_layout()
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

# -----------------------------
# 5) Ejecutar
# -----------------------------
if __name__ == "__main__":
    os.makedirs("Datos", exist_ok=True)
    df_sim = simulate_thermal_stress_in_cattle_pen_aligned()
    df_sim.to_csv("Datos/thermal_stress_simulation.csv", index=False)
    print("Dataset guardado en Datos/thermal_stress_simulation.csv")

    plot_temp_hum_co2(df_sim, out_path="Resultados/fig2_combined.png")
    print("Figura combinada guardada en Resultados/fig2_combined.png")

    plot_temperature_only(df_sim, out_path="Resultados/fig4_temperature_only.png")
    print("Figura de temperatura (Fig. 4) guardada en Resultados/fig4_temperature_only.png")

Dataset guardado en Datos/thermal_stress_simulation.csv
Figura combinada guardada en Resultados/fig2_combined.png
Figura de temperatura (Fig. 4) guardada en Resultados/fig4_temperature_only.png
